In [ ]:
import numpy as np
import csv
import pandas as pd

In [ ]:
df = pd.read_csv("aqburk-20190301T000000Z.csv")
df.head()


In [ ]:
df = df.sort_values(by = ['dev-id', 'readable_time'] )#sorting by id AND time
print(df.shape)
df.head()

In [ ]:
print(list(df))
#We are only interested in pm10avg. pm10avg stands for average particulate matter, in µg/ m3 air. 

In [ ]:
def filter_columns(df, keep):
	"""Filter Pandas table df, keeping only columns in keep"""
	allColumns = list(df)
	for col in keep:
		allColumns.remove(col)

	return df.drop(columns=allColumns)

keep_columns = ['time', 'readable_time', 'humi', 'pm10avg', 'pres', 'rssi', 'temp', 'dev-id']
dfpm10 = filter_columns(df, keep_columns)
dfpm10.head()



In [ ]:
dfpm10['dev-id'].value_counts()


In [ ]:
#We are only interested in the sensor with dev-id: 373773207E330103
#get index of the first 373773207E330103
for a in range (0, len(dfpm10)):
    if (dfpm10['dev-id'].iloc[a] == "373773207E330103"): 
        print(a)
        break
       

In [ ]:
#get index of the first 373773207E330104 (after 373773207E330103)
for b in range (a, len(dfpm10)):
    if (dfpm10['dev-id'].iloc[b] == "373773207E330104"): 
        print(b)
        break


In [ ]:
dfSensor = dfpm10[a:b]
dfSensor.head()


In [ ]:
dfSensor.tail()

In [ ]:
#making sure or sensor is the right one 
dfSensor['dev-id'].value_counts()


In [ ]:
dfSensor.readable_time.describe()

In [ ]:
#We check the type of readable_time
type(dfSensor.readable_time)

In [ ]:
#slow, transforming time to correct readable format
'''
Our time is given in format: 2019-02-28T11:45:33.239000Z.
In datetime.strptime this translate to : %Y-%m-%dT%H:%M:%S.%fZ".

Not all time is given in the correct format. We want to use time in the format: 2019-02-28 11:45:33, 
or %Y-%m-%dT%H:%M:%SZ in datetime.strptime. We are only interested in elements 0..19 of our readable_time array.


'''
import datetime
import time

for n in range (0, len(dfSensor)):
    date_time_str = dfSensor.readable_time.iloc[n]
    temp = date_time_str[:19]
    date_time_obj = datetime.datetime.strptime(temp, '%Y-%m-%dT%H:%M:%S') 
    dfSensor.readable_time.iloc[n] = date_time_obj
    
dfSensor.tail()

In [ ]:
#dfSensor2 = dfSensor #Safety copy of dfSensor

In [ ]:
keep_columns = ['readable_time', 'pm10avg']
dfSensor = filter_columns(dfSensor, keep_columns)
print(dfSensor.shape)
dfSensor.tail()


In [ ]:
dfSensor = dfSensor.set_index(dfSensor.readable_time)
dfSensor.tail()


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

def plot_figure(readable_time, pm10avg):
    plt.figure(figsize=(40,20))
    plt.plot(readable_time, pm10avg)
    plt.xlabel("Date")
    plt.ylabel("PM10, µg/m3 ")

plot_figure(dfSensor.readable_time, dfSensor.pm10avg)
plt.title("PM10 average as a function of time")
#plt.savefig('RawGraph.png', dpi=80)
plt.show()


In [ ]:
#resample, calculate hourly average for pm10
hourly = dfSensor.resample('H').mean()
hourly.tail()

In [ ]:
plot_figure(hourly.index, hourly.pm10avg)
plt.title("pm10avg, calculated as 1 h mean")
#plt.savefig('RawGraph1HourAverage.png', dpi=80)
plt.show()

# Removing Spikes

Peaks will affect and strongly influence the mean hourly value, whereas (single) spikes are hardly recognised for hourly values.

How can we remove single, short time peaks (spikes) from the data? How do we define single, short time peaks?

My answer is to replace or remove the spikes and replace them with an averaged values. Spikes are values above a threshold value, and they are replaced with the mean of the previous and the next value.

We will iterate over our data three times. By this method, spikes will be removed or at least weakend, whereas peaks will only weaken. 


In [ ]:
#evaluate with a smaller area, to investigate our spike reducing method
df1 = dfSensor[500:1000]
df1.tail()

In [ ]:
plot_figure(df1.readable_time, df1.pm10avg)
plt.title("pm10avg as function of time, ")
#plt.savefig('RawGraph1Reduced.png', dpi=80)
plt.show()


In [ ]:
df1 = df1.reset_index(drop=True)
df1.tail()

In [ ]:
threshold = 20
count = 0
#I will iterate 3 times over my data to remove/ or smooth down spikes

for m in range (0,3):
    for n in range (0,(len(df1)-1)):
        if (df1.pm10avg.iloc[n] > threshold):
            prev = df1.pm10avg.iloc[n-1]
            nxt = df1.pm10avg.iloc[n+1]
            avg = ((prev+nxt)/2)
            df1.pm10avg.iloc[n] = avg
            count = count + 1

print("Nr of values changed: ", count)

In [ ]:
plot_figure(df1.readable_time, df1.pm10avg)
plt.title("pm10avg as function of time, manipulated, threshold 20")
#plt.savefig('RawGraph1ReducedWithThreshold.png', dpi=80)
plt.show()

In [ ]:
plot_figure(dfSensor.readable_time, dfSensor.pm10avg)
plt.title("PM10 average as a function of time/ 2nd time")
plt.show()


In [ ]:
dfSensor = dfSensor.reset_index(drop=True)
dfSensor.tail()

In [ ]:
threshold = 20
count = 0

for m in range (0,3):
    for n in range (0,(len(dfSensor)-1)):
        if (dfSensor.pm10avg.iloc[n] > threshold):
            prev = dfSensor.pm10avg.iloc[n-1]
            nxt = dfSensor.pm10avg.iloc[n+1]
            avg = ((prev+nxt)/2)
            dfSensor.pm10avg.iloc[n] = avg
            count = count + 1

#gives nr of non-churned, poor names
print("Nr of values changed: ", count, "n: ",n)

In [ ]:
plot_figure(dfSensor.readable_time, dfSensor.pm10avg)
plt.title("pm10avg as function of time/spikes reduced")
#plt.savefig('RawGraph1SpikesReduced.png', dpi=80)
plt.show()

In [ ]:
dfSensor = dfSensor.set_index(dfSensor.readable_time)
dfSensor.tail()

Resampling involves changing the frequency of your time series observations. One reason why you may be interested in resampling your time series data is feature engineering. 
Indeed, it can be used to provide additional structure or insight into the learning problem for supervised learning models. 



In [ ]:
hourly = dfSensor.resample('H').mean()
hourly.tail()

In [ ]:
plot_figure(hourly.index, hourly.pm10avg)
plt.title("pm10avg/ Spikes reduced/ 1 h mean")
#plt.savefig('RawGraph1SpikesReducedHourlyAverage.png', dpi=80)
plt.show()

In [ ]:
#Is there more polutions on a specific weekday?
#Do our baseline change with time?

daily = dfSensor.resample('D').mean()
daily


In [ ]:
plot_figure(daily.index, daily.pm10avg)
plt.title("pm10avg/ Spikes reduced/ daily mean")
#plt.savefig('RawGraph1SpikesReducedDailyAverage.png', dpi=80)
plt.show()

We'll add hour, day of week, and a boolean for is_weekend. This will expand our possibilities for feature analysis. 

Fetaure analysyis: hour, day, weekend/ weekday


In [ ]:
hourly["hour"] = hourly.index.hour
hourly["weekday"] = hourly.index.weekday
hourly['is_weekend'] = hourly.weekday.isin([5,6])*1
print(hourly.shape)
hourly.tail()


In [ ]:
#Say we just want to see data where the hour is 2, we could use the index as per below.
by_hour2 = hourly [hourly.index.hour == 2]
by_hour2

In [ ]:
by_hour = hourly.groupby(hourly.hour).mean()#calculates mean for pm10avg
print(by_hour.shape)
by_hour

In [ ]:
plot_figure(by_hour.index, by_hour.pm10avg)
plt.title("pm10avg as function of hour of the day 15-28.2.2019")
#plt.savefig('HourlyAverage.png', dpi=80)
plt.show()


In [ ]:
by_weekday = hourly.groupby(hourly.weekday).mean()
print(by_weekday.shape)
by_weekday

In [ ]:
plot_figure(by_weekday.index, by_weekday.pm10avg)
plt.title("pm10avg as function of weekday 15-28.2.2019")
#plt.savefig('WeekdayAverage.png', dpi=80)
plt.show()


In [ ]:
by_weekend = hourly.groupby(hourly.is_weekend).mean()
print(by_weekend.shape)
by_weekend